In [4]:
def img_process(img):
    clone = img.copy()
    
    #灰階
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #sobel邊緣偵測
    x = cv2.Sobel(gray, cv2.CV_16S, 1, 0)      # 偵測邊緣
    y = cv2.Sobel(gray, cv2.CV_16S, 0, 1)      # 偵測邊緣
    absx=cv2.convertScaleAbs(x)
    absy=cv2.convertScaleAbs(y)
    dst=cv2.addWeighted(absx,0.8,absy,0.2,0)
    
    # 對灰度圖像進行二值化
    ret, thresh = cv2.threshold(dst, 200, 255, cv2.THRESH_BINARY)
    
    # 對二值化圖像進行形態學處理
    kernelx = cv2.getStructuringElement(cv2.MORPH_RECT, (50, 1))
    kernely = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 30))

    morph = cv2.dilate(thresh, kernelx)
    morph = cv2.erode(morph, kernelx)
    
    morph = cv2.erode(morph, kernely)
    morph = cv2.dilate(morph, kernely)
    
    # 抓取車牌輪廓
    contours, hierarchy = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]
    max_weight=0
    for item in contours:
        x,y,weight,height = cv2.boundingRect(item)
    #     print(x,y,weight,height)    
        if (weight / height > 1.5) and (weight>max_weight):
            max_weight=weight
            crop_img = img[y - 15:y + height + 10, x:x + weight+5]    
            cv2.rectangle(clone,(x,y),(x + weight,y + height),(0,0,255),2)
      
        
    return clone        
            

In [7]:
def Detect(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #convert to grey scale
    blur_img = cv2.bilateralFilter(gray, 11, 17, 17)
    ret, th1 = cv2.threshold(blur_img,165,255,cv2.THRESH_BINARY)
    ret, th2 = cv2.threshold(gray,165,255,cv2.THRESH_BINARY)

    kernelx = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 1))
    kernely = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 3))
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 1))

    morph = cv2.dilate(th1, kernelx)
    morph = cv2.erode(morph, kernelx)
    morph = cv2.erode(morph, kernely)
    morph = cv2.dilate(morph, kernely)
    #morph = cv2.erode(morph, kernel)
    morph = cv2.dilate(morph, kernel)

    edged = cv2.Canny(gray, 30, 180) #Perform Edge detection
    edged1 = cv2.Canny(morph, 30, 200) #Perform Edge detection

    cnts, _ = cv2.findContours(morph.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    clone = img.copy()
#     cv2.drawContours(clone, cnts, -1, (0, 255, 0),1)

    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:10]
    max_weight = 0

    for item in cnts:
        x,y,weight,height = cv2.boundingRect(item)   
        if (weight / height > 1.5) and (weight / height < 3) and(weight>max_weight):
            max_weight=weight
            crop_img = morph[y+25:y + height-30, x+10:x + weight-10]
            cv2.rectangle(clone,(x,y),(x + weight,y + height),(0,0,255),2)
            
#     img_output = cv2.cvtColor(clone, cv2.COLOR_BGR2RGB)
    
    return clone

In [8]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
import cv2
import numpy as np
import time

base = BaseOverlay("base.bit")

Mode = VideoMode(640, 480, 24)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode, PIXEL_BGR)
hdmi_out.start()

# camera (input) configuration
frame_in_w = 640
frame_in_h = 480

videoIn = cv2.VideoCapture(0)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w)
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h)
print("capture device is open: " + str(videoIn.isOpened()))

# num_frames = 1500
readError = 0

while True:
    # read next image
    start = time.time()
    ret, frame_vga = videoIn.read()

    if ret:
        # 將彩色幀轉換為灰度
        gray_frame = img_process(frame_vga)
        
        # 將灰度幀輸出到HDMI
        outframe = hdmi_out.newframe()
        outframe[:] = gray_frame
        hdmi_out.writeframe(outframe)
    else:
        readError += 1

    end = time.time()
    seconds = end - start
#     num_frames -= 1
    if(base.buttons[0].read()):
        break
    
# 停止視訊輸入和輸出
hdmi_out.stop()
print("hdmi_out.stop")
# 釋放資源
videoIn.release()
print("videoIn.release")
base.free()
print("base.free()")
cv2.destroyAllWindows()
print("cv2.destroyAllWindows()")

[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (616) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


capture device is open: True
hdmi_out.stop
videoIn.release
base.free()
cv2.destroyAllWindows()
